# 1. Calibrate VasicekModel parameter 

Estimation of Parameters: mle, ls, moment matching 

In [1]:
# Import dependencies
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import typing
from datetime import date, timedelta, datetime
import sys

sys.path.append("../utils")
from utils import *
from interest_rate import *
from sklearn.linear_model import LinearRegression

In [3]:
def calibrate_vasicek_model_least_squares(bond_data, initial_params):
    """
    Calibrate the Vasicek interest rate model using the least squares method.
    """

    def vasicek_objective(params):
        """
        Objective function for Vasicek model parameter optimization.
        """
        r, k, theta = params
        model_prices = []
        for _, row in bond_data.iterrows():
            t = row["period"]
            model_price = row["Price"] * np.exp(-r * t)
            model_prices.append(model_price)
        return np.mean((bond_data["Price"] - model_prices) ** 2)

    result = minimize(
        vasicek_objective,
        initial_params,
        method="L-BFGS-B",
        bounds=[(0, 1), (0, 1), (0, 1)],
    )
    r, k, theta = result.x
    return r, k, theta


def calibrate_vasicek_model_mle(bond_data, initial_params):
    """
    Calibrate the Vasicek interest rate model using maximum likelihood estimation.
    """

    def vasicek_likelihood(params):
        """
        Log-likelihood function for the Vasicek model.
        """
        r, k, theta = params
        likelihood = 0
        for i in range(1, len(bond_data)):
            t = bond_data["period"].iloc[i]
            r_t = bond_data["Price"].iloc[i]
            r_t_1 = bond_data["Price"].iloc[i - 1]
            likelihood += np.log(
                np.exp(-(r_t - r_t_1 * np.exp(-k * t)) / (1 - np.exp(-k * t)) * theta)
                / (t * theta * np.sqrt(1 - np.exp(-2 * k * t)))
            )
        return -likelihood

    result = minimize(
        vasicek_likelihood,
        initial_params,
        method="L-BFGS-B",
        bounds=[(0, 1), (0, 1), (0, 1)],
    )
    r, k, theta = result.x
    return r, k, theta


def calibrate_vasicek_model_moment_matching(bond_data):
    """
    Calibrate the Vasicek interest rate model using moment matching.
    """
    X = bond_data["period"].to_numpy()
    Y = bond_data["Price"].to_numpy()

    # Fit a linear regression model to the log-transformed bond prices
    model = LinearRegression()
    model.fit(X.reshape(-1, 1), np.log(Y))
    k = -model.coef_[0]
    theta = np.sqrt(model.score(X.reshape(-1, 1), np.log(Y)) * k)
    r = np.mean(Y) / np.mean(np.exp(-X * k))

    return r, k, theta


def main():
    data_dir = "../data/bond/"
    bond_data = read_bond_data(data_dir)

    # Calculate initial parameter estimates from historical data
    r_initial = np.mean(bond_data["Price"])
    k_initial = (
        -LinearRegression()
        .fit(
            bond_data["period"].to_numpy().reshape(-1, 1),
            np.log(bond_data["Price"].to_numpy()),
        )
        .coef_[0]
    )
    theta_initial = np.sqrt(
        LinearRegression()
        .fit(
            bond_data["period"].to_numpy().reshape(-1, 1),
            np.log(bond_data["Price"].to_numpy()),
        )
        .score(
            bond_data["period"].to_numpy().reshape(-1, 1),
            np.log(bond_data["Price"].to_numpy()),
        )
        * k_initial
    )
    initial_params = [r_initial, k_initial, theta_initial]

    print("Least Squares Calibration:")
    r_ls, k_ls, theta_ls = calibrate_vasicek_model_least_squares(
        bond_data, initial_params
    )
    print(f"r: {r_ls:.4f}, k: {k_ls:.4f}, θ: {theta_ls:.4f}")

    print("\nMaximum Likelihood Estimation:")
    r_mle, k_mle, theta_mle = calibrate_vasicek_model_mle(bond_data, initial_params)
    print(f"r: {r_mle:.4f}, k: {k_mle:.4f}, θ: {theta_mle:.4f}")

    print("\nMoment Matching:")
    r_mm, k_mm, theta_mm = calibrate_vasicek_model_moment_matching(bond_data)
    print(f"r: {r_mm:.4f}, k: {k_mm:.4f}, θ: {theta_mm:.4f}")


if __name__ == "__main__":
    main()

Least Squares Calibration:
r: 0.0000, k: 0.0934, θ: 0.0588

Maximum Likelihood Estimation:


/var/folders/km/_b5m00hj0l91hg5vzm9rf92m0000gn/T/ipykernel_4103/1807510469.py:42: RuntimeWarning: divide by zero encountered in scalar divide
  likelihood += np.log(np.exp(-(r_t - r_t_1 * np.exp(-k * t)) / (1 - np.exp(-k * t)) * theta) / (t * theta * np.sqrt(1 - np.exp(-2 * k * t))))
/var/folders/km/_b5m00hj0l91hg5vzm9rf92m0000gn/T/ipykernel_4103/1807510469.py:42: RuntimeWarning: invalid value encountered in scalar multiply
  likelihood += np.log(np.exp(-(r_t - r_t_1 * np.exp(-k * t)) / (1 - np.exp(-k * t)) * theta) / (t * theta * np.sqrt(1 - np.exp(-2 * k * t))))
/var/folders/km/_b5m00hj0l91hg5vzm9rf92m0000gn/T/ipykernel_4103/1807510469.py:42: RuntimeWarning: invalid value encountered in scalar divide
  likelihood += np.log(np.exp(-(r_t - r_t_1 * np.exp(-k * t)) / (1 - np.exp(-k * t)) * theta) / (t * theta * np.sqrt(1 - np.exp(-2 * k * t))))


r: 1.0000, k: 0.0934, θ: 0.0588

Moment Matching:
r: 1.3914, k: 0.0934, θ: 0.0588


In [ ]:
Choose